In [14]:
import os
import pandas as pd
from ete3 import Tree
from collections import Counter
import numpy as np
import itertools

In [7]:
os.getcwd()
os.chdir('/Volumes/AHN/captive_ape_microbiome/results/gyrb/')

In [8]:
#inputs
metadata_file = 'inputs/metadata_gyrb_amp_meta_passing_samples.txt'
tax_table_file = 'inputs/ASVs_taxonomy.txt'
asv_table_file = 'inputs/ASVs_filtered_counts.tsv'
asv_fasta_file = 'inputs/ASVs_filtered.fasta'
full_tree_file = 'inputs/ASVs_filtered_ref_full.tree'
#read tree with ete3 
full_tree = Tree(full_tree_file, format=0)
moeller_codiv_fasta = 'inputs/codiv_Bacteroidaceae.fna'
#set blast percent identity cutoff 
pident_cutoff = 95

In [9]:
asv_table = pd.read_csv(asv_table_file,sep='\t',index_col=0)

sampleNames = asv_table.apply(lambda row: list(row.index[row>0]),axis=1)
ASV_sampleName_dict = dict(zip(sampleNames.index,sampleNames))

#sample to sample type category
metadata = pd.read_csv(metadata_file,sep='\t',index_col=None)
sample_type_dict = dict(zip(metadata['X.SampleID'], metadata['Description']))

#taxonomic info, family and genus
tax_table = pd.read_csv(tax_table_file,sep='\t',index_col=None)
tax_fam_dict = dict(zip(tax_table['ASV'], tax_table['Family']))
tax_gen_dict = dict(zip(tax_table['ASV'], tax_table['Genus']))
    

In [10]:
%%bash

mkdir analyses/codiv_moeller_ASVs
cp inputs/moeller_codiv_Bacteroidaceae.fna analyses/codiv_moeller_ASVs/moeller_codiv_Bacteroidaceae.fna

#make blastdb
makeblastdb -in analyses/codiv_moeller_ASVs/moeller_codiv_Bacteroidaceae.fna -dbtype nucl

#blast moeller co-div seqs
blastn -query inputs/ASVs_filtered.fasta -db analyses/codiv_moeller_ASVs/moeller_codiv_Bacteroidaceae.fna \
-outfmt "7 qseqid salltitles sseqid pident length qlen evalue" \
-out analyses/codiv_moeller_ASVs/codiv_blastout_ASVs.txt \
-max_target_seqs 5



Building a new DB, current time: 09/09/2020 09:53:01
New DB name:   /Volumes/AHN/captive_ape_microbiome/results/gyrb/analyses/codiv_moeller_ASVs/moeller_codiv_Bacteroidaceae.fna
New DB title:  analyses/codiv_moeller_ASVs/moeller_codiv_Bacteroidaceae.fna
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Volumes/AHN/captive_ape_microbiome/results/gyrb/analyses/codiv_moeller_ASVs/moeller_codiv_Bacteroidaceae.fna
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 208 sequences in 0.009305 seconds.


mkdir: analyses/codiv_moeller_ASVs: File exists


In [12]:
#filter blast output to top hit per ASV, remove hits not pass thresholds
blast_res = pd.read_csv('analyses/codiv_moeller_ASVs/codiv_blastout_ASVs.txt',sep='\t',comment='#',header=None)
blast_res.columns = ['ASV','codiv_clade_seq','sseqid','pident','length','qlen','evalue']
blast_res = blast_res.groupby('ASV').head(1).reset_index(drop=True) #get top hit
blast_res_pass = blast_res[blast_res['pident']>pident_cutoff].reset_index(drop=True) #QC filter
blast_res_pass = blast_res_pass[blast_res_pass['length']>200].reset_index(drop=True) #QC filter
print(len(blast_res_pass),'ASVs matching co-div clades with greater than',pident_cutoff,'percent identity')
blast_res_pass['codiv_clade'] = blast_res_pass['codiv_clade_seq'].apply(lambda x: x.split(' ')[1])
blast_res_pass['lineage'] = blast_res_pass['codiv_clade'].apply(lambda x: x.split('_')[0]) 
blast_res_pass = blast_res_pass[['ASV','codiv_clade_seq', 'codiv_clade','lineage']]
#print(blast_res_pass.head())
print(blast_res_pass['codiv_clade'].value_counts())
#output hits passing that will be used to determine the mcra of the lineage
blast_res_pass.to_csv('analyses/codiv_moeller_ASVs/codiv_clades_ASVs.txt',sep='\t',index=False)


314 ASVs matching co-div clades with greater than 95 percent identity
Bt2_clade1_bonobo     90
Bt3_clade1_human      85
Bt2_clade2_bonobo     33
Bt1_clade1_bonobo     23
Bt2_clade1_chimp      20
Bt3_clade1_chimp      17
Bt3_clade1_bonobo     15
Bt1_clade1_gorilla    10
Bt1_clade1_chimp       8
Bt2_clade1_gorilla     7
Bt2_clade2_chimp       6
Name: codiv_clade, dtype: int64


# defining host restricted clades

In [31]:
def is_HR(sampleNames,sample_type_dict):
    """given a list of sample names uses sample type dictionary to determine how many sample types are present
    designates ASVs as host restricted = 1 sample type or mixed = multiple sample types.
    Captive sample types are not considered so some clades will have a 0 sample type length and they can fall within
    host-restricted clades or mixed clades or neither"""
    sampleTypes = [sample_type_dict[name] for name in sampleNames]
    sampleTypes = [x.replace('non_western_','').replace('western_','') for x in sampleTypes]
    neutral_sampleTypes = ['captive_gorilla','captive_bonobo','captive_chimp','captive_orangutan']
    HR_sampleTypes = list(set(sampleTypes) - set(neutral_sampleTypes))
    HR_sampleNum = len([x for x in sampleTypes if x not in neutral_sampleTypes])
    CP_sampleTypes = list(set(sampleTypes) & set(neutral_sampleTypes))
    CP_sampleNum = len([x for x in sampleTypes if x in neutral_sampleTypes])
    CP_pres = True if len(CP_sampleTypes) > 0 else False
    if len(HR_sampleTypes) == 0:
        HR_cat,HR_type='CP','CP'
    if len(HR_sampleTypes) == 1: #identifies host-restricted clades
        HR_cat,HR_type='HR',HR_sampleTypes[0]  
    if len(HR_sampleTypes) > 1:    
        HR_cat,HR_type ='MX','MX'
    return(HR_sampleTypes,HR_sampleNum,HR_cat,HR_type,CP_pres,CP_sampleTypes,CP_sampleNum)

def is_HR(sampleNames,sample_type_dict):
    """given a list of sample names uses sample type dictionary to determine how many sample types are present
    designates ASVs as host restricted = 1 sample type or mixed = multiple sample types.
    Captive sample types are not considered so some clades will have a 0 sample type length and they can fall within
    host-restricted clades or mixed clades or neither"""
    sampleTypes = [sample_type_dict[name] for name in sampleNames]
    sampleTypes = [x.replace('non_western_','').replace('western_','') for x in sampleTypes]
    neutral_sampleTypes = ['captive_gorilla','captive_bonobo','captive_chimp','captive_orangutan']
    HR_sampleTypes = list(set(sampleTypes) - set(neutral_sampleTypes))
    HR_sampleNum = len([x for x in sampleTypes if x not in neutral_sampleTypes])
    CP_sampleTypes = list(set(sampleTypes) & set(neutral_sampleTypes))
    CP_sampleNum = len([x for x in sampleTypes if x in neutral_sampleTypes])
    CP_pres = True if len(CP_sampleTypes) > 0 else False
    if len(HR_sampleTypes) == 0:
        HR_cat,HR_type='Unique_CP','Unique_CP'
    if len(HR_sampleTypes) == 1: #identifies host-restricted clades
        HR_cat,HR_type='HR','HR_'+HR_sampleTypes[0] 
    if len(HR_sampleTypes) > 1:    
        HR_cat='MX'
        if len(HR_sampleTypes) == 2:
            if 'human' in HR_sampleTypes:
                HR_type = 'MX_human_single_wild_ape'
            else:
                HR_type = 'MX_2_wild_apes'
        if len(HR_sampleTypes) == 3:
            if 'human' in HR_sampleTypes:
                HR_type = 'MX_human_2_wild_apes'
            else:
                HR_type = 'MX_3_wild_apes'
        if len(HR_sampleTypes) == 4:
            HR_type = 'MX_4_hominids'
            
    return(HR_sampleTypes,HR_sampleNum,HR_cat,HR_type,CP_pres,CP_sampleTypes,CP_sampleNum)

    #if len(HR_sampleTypes) == 0:
    #    HR_cat,HR_type='Unique_CP','Unique_CP'
   # if len(HR_sampleTypes) == 1: #identifies host-restricted clades
   #     HR_cat,HR_type='HR','HR_'+HR_sampleTypes[0]  
    #if len(HR_sampleTypes) > 1: 
    #    HR_cat = 'NHR'
    #    if len(HR_sampleTypes) == 2:
     #       if 'human' in HR_sampleTypes:
     #           HR_type = 'NHR_human_single_wild_ape'
     #       else:
    #            HR_type = 'NHR_2_wild_apes'
    #    if len(HR_sampleTypes) == 3:
    #        if 'human' in HR_sampleTypes:
    #            HR_type = 'NHR_human_2_wild_apes'
    #        else:
    #            HR_type = 'NHR_3_wild_apes'
    #    if len(HR_sampleTypes) == 4:
    #        HR_type = 'NHR_4_hominids'
        
    #return(HR_sampleTypes,HR_sampleNum,HR_cat,HR_type,CP_pres,CP_sampleTypes,CP_sampleNum)
           
           
print(is_HR(['CosteaPI_2017__SID713A076-11-0-0','LiSS_2016__FAT_024-22-42-0', 
       'LiSS_2016__FAT_024-22-84-0', 'LiSS_2016__FAT_DON_11-22-0-4'],sample_type_dict)) 

def get_consensus_taxonomy(listASVs,tax_fam_dict,tax_gen_dict):
    """Because multiple ASVs in a single clade, we determine the taxonomy of the clade by consensus.
    If all ASVs in the clade belong to a given bacterial taxonomic family/genus the clade is assigned that taxonomy
    If the clade contains ASVs assigned to multiple families/genera the taxonomy is labeled as unassigned."""
    fam = list(set([tax_fam_dict[ASV] for ASV in listASVs])-set(['Unassigned']))
    gen = list(set([tax_gen_dict[ASV] for ASV in listASVs])-set(['Unassigned']))
    fam = fam[0].split('__')[1] if len(fam) == 1 else 'Unassigned'
    if fam == 'Unassigned':
        gen='Unassigned'
    else:
        gen = gen[0].split('__')[1] if len(gen) == 1 else 'Unassigned'  
    return(fam+'_'+gen)   
print(get_consensus_taxonomy(['ASV_1','ASV_2','ASV_1078'],tax_fam_dict,tax_gen_dict))

def search_clades(tree, samples_cutoff, BS_support,ASV_sampleName_dict):
    """Finds nodes with at least 50% BS support containing ASVs only found in a single wild ape species
    ie wild_gorilla, wild_chimp or wild_bonobo, cpat"""
    clades_prelim = []
    counter = 1
    for n in tree.traverse():
        if n.support > float(BS_support): #makes sure Bootstrap support is over threshold
            ASVs = [leaf.name for leaf in n.iter_leaves() if 'ASV' in leaf.name]
            listoflists = [ASV_sampleName_dict[ASV] for ASV in ASVs]
            sampleNames = list(set(list(itertools.chain.from_iterable(listoflists))))
            cladeName='clade_'+str(counter)
            counter+=1
            clade = [cladeName,ASVs,sampleNames]
            if len(sampleNames)>samples_cutoff:
                clades_prelim.append(clade)
    clades_prelim = pd.DataFrame(clades_prelim, columns = 
                          ['cladeName','ASVs','sampleNames'])
    clades_prelim['sampleNum'] = clades_prelim['sampleNames'].apply(lambda x: len(x))
    clades_prelim['ASVsNum'] = clades_prelim['ASVs'].apply(lambda x: len(x))
    return(clades_prelim)

def eliminate_redundant_clades(clades_df,offlimits_ASVs):
    """sorts clades and returns the largest non overlapping clade """
    df = clades_df.sort_values('ASVsNum',ascending=False) #start with the largest clades first
    NRclades = []
    for index, row in df.iterrows():
        if len(set(row['ASVs']) & set(offlimits_ASVs)) == 0: 
            offlimits_ASVs = offlimits_ASVs + row['ASVs']
            NRclades.append(row['cladeName']) 
    res = df[df['cladeName'].isin(NRclades)]   
    return(res)  

def expand_clades(clades_df):
    ASV = clades_df.apply(lambda x: pd.Series(x['ASVs']),axis=1).stack().reset_index(level=1, drop=True)
    ASV.name = 'ASVs'
    clades_ASVs_df = clades_df.drop('ASVs', axis=1).join(ASV)
    return(clades_ASVs_df)
                                 
def host_restricted_clades(asv_table_file,metadata_file,tax_table_file):
    asv_table = pd.read_csv(asv_table_file,sep='\t',index_col=0)
    sampleNames = asv_table.apply(lambda row: list(row.index[row>0]),axis=1)
    ASV_sampleName_dict = dict(zip(sampleNames.index,sampleNames))

    #sample to sample type category
    metadata = pd.read_csv(metadata_file,sep='\t',index_col=None)
    sample_type_dict = dict(zip(metadata['X.SampleID'], metadata['Description']))

    #taxonomic info, family and genus
    tax_table = pd.read_csv(tax_table_file,sep='\t',index_col=None)
    tax_fam_dict = dict(zip(tax_table['ASV'], tax_table['Family']))
    tax_gen_dict = dict(zip(tax_table['ASV'], tax_table['Genus']))
    
    #search tree for clades
    clades_prelim = search_clades(full_tree, 5, .5, ASV_sampleName_dict)
    hr =clades_prelim['sampleNames'].apply(lambda x:  pd.Series(is_HR(x,sample_type_dict),
                index=['HR_sampleTypes','HR_sampleNum','HR_cat','HR_type','CP_pres','CP_sampleTypes','CP_sampleNum']))
    clades_prelim_hr = clades_prelim.merge(hr,right_index=True,left_index=True)

    #identify HR clades
    HR_clades = clades_prelim_hr[clades_prelim_hr['HR_cat']=='HR']
    HR_clades = HR_clades[HR_clades['HR_sampleNum']>5]
    HR_clades = eliminate_redundant_clades(HR_clades,[])
    HR_clades_ASVs = expand_clades(HR_clades)

    #identify MX clades that don't contain any HR clades
    MX_clades = clades_prelim_hr[clades_prelim_hr['HR_cat']=='MX']
    MX_clades = MX_clades[MX_clades['sampleNum']>5]
    MX_clades = eliminate_redundant_clades(MX_clades,offlimits_ASVs=list(HR_clades_ASVs['ASVs']))
    MX_clades_ASVs = expand_clades(MX_clades)
    
    #identify MX clades that don't contain any HR clades
    CP_clades = clades_prelim_hr[clades_prelim_hr['HR_cat']=='Unique_CP']
    CP_clades = CP_clades[CP_clades['sampleNum']>5]
    HR_MX_ASVs = list(HR_clades_ASVs['ASVs']) + list(MX_clades_ASVs['ASVs'])
    CP_clades = eliminate_redundant_clades(CP_clades,offlimits_ASVs=HR_MX_ASVs)
    CP_clades_ASVs = expand_clades(CP_clades)
    
    #merge dataframes
    clades = pd.concat([HR_clades,MX_clades,CP_clades])
    clades.reset_index(drop=True, inplace=True)
    clades_ASVs = pd.concat([HR_clades_ASVs,MX_clades_ASVs,CP_clades_ASVs])
    clades_ASVs.reset_index(drop=True, inplace=True)
    
    
    clades['cladeTax']=clades['ASVs'].apply(lambda x: 
        get_consensus_taxonomy(x,tax_fam_dict,tax_gen_dict))
    
    sample_type_counts =  metadata['Description'].value_counts()
    description_df = clades['sampleNames'].apply(lambda l: pd.Series(
    [sample_type_dict[name] for name in l]).value_counts())
    description_df = description_df.fillna(0)  
    sample_type_percent = description_df/sample_type_counts
    clades=clades.merge(sample_type_percent,left_index=True,right_index=True)  
    
    return(clades,clades_ASVs)
    


(['human'], 4, 'HR', 'HR_human', False, [], 0)
Bacteroidaceae_Unassigned


## Run function on gyrb data

In [ ]:
clades_df,clades_ASVs_df = host_restricted_clades(asv_table_file,metadata_file,tax_table_file)   

In [48]:
print(clades_df.shape)
#print(clades_df_sh.head())

#Used for Figure3 tips on tree are ASVs, not clades
print(clades_ASVs_df.shape)
#print(clades_ASVs_df.head())

print('How many of the clades are present in captive apes?')
captive_clades = clades_df[clades_df['CP_pres']==True]
print(len(captive_clades))
print('Are these clades host-restricted, mixed host, or unique to captive?')
print(captive_clades['HR_cat'].value_counts())


def just_captive_samples(sampleNames):
    return([name for name in sampleNames if 'captive' in sample_type_dict[name]])    
clades_df['captiveNames'] = clades_df['sampleNames'].apply(lambda x: just_captive_samples(x))  

prominent_clades = clades_df.loc[(clades_df.captive_bonobo > 0.15) | 
              (clades_df.captive_chimp > 0.15)  |
              (clades_df.captive_gorilla > 0.15)|
              (clades_df.captive_orangutan > 0.15)]
clades_df['CP_prominent'] = clades_df['cladeName'].apply(lambda x: x in list(prominent_clades['cladeName']))
print(clades_df['CP_prominent'].value_counts())   

#clades_df_sh = clades_df.drop(['ASVs','sampleNames'], axis=1)
clades_df_sh = clades_df[['cladeName', 'cladeTax', 'sampleNum', 'ASVsNum', 
      'HR_sampleTypes', 'HR_sampleNum','HR_cat', 'HR_type', 
      'CP_pres','CP_prominent', 'CP_sampleTypes', 'CP_sampleNum','captiveNames',
       'captive_bonobo', 'captive_chimp', 'captive_gorilla',
       'captive_orangutan', 'non_western_human', 'western_human',
       'wild_bonobo', 'wild_chimp', 'wild_gorilla']]
       

clades_df_sh.to_csv('analyses/codiv_moeller_ASVs/HRclades_Figure2_table.txt',sep='\t',index=False)

(359, 25)
(6438, 12)
How many of the clades are present in captive apes?
52
Are these clades host-restricted, mixed host, or unique to captive?
HR           27
MX           19
Unique_CP     6
Name: HR_cat, dtype: int64
False    328
True      31
Name: CP_prominent, dtype: int64


### search full tree for host-restricted clades

In [49]:
collasped_tree = Tree(full_tree_file, format=0)

def collapse_node(tree,clade_ASVs,cladeName):
    if len(clade_ASVs) > 1:
        node = tree.get_common_ancestor(clade_ASVs)
        node.name = cladeName
        children = node.get_children()
        node.remove_child(children[1])
        node.remove_child(children[0])
    else:
        ASV = clade_ASVs[0]
        leaf = tree.get_leaves_by_name(name=ASV)[0]
        leaf.name = cladeName

for clade,row in clades_df.iterrows():
    collapse_node(collasped_tree,row['ASVs'],row['cladeName'])

clade_leaves = [leaf for leaf in collasped_tree.get_leaves() if 'clade' in leaf.name] 
collasped_tree.prune(clade_leaves) #eliminate leftover ASVs and ref taxa
collasped_tree.write(format=2, outfile='analyses/codiv_moeller_ASVs/HRclades_Figure2.tre')


# ASV info for subtree figures

In [51]:
allASVs = [leaf.name for leaf in full_tree.get_leaves() if 'ASV' in leaf.name]
allASVs = pd.DataFrame(allASVs,columns=['ASV'])
print(len(allASVs))

#Host restricted clades 
print(clades_ASVs_df.head())
print(len(clades_ASVs_df))
#Moeller codiv clades
print(blast_res_pass.head())
print(len(blast_res_pass))

#combine
allASVs_HRclade = allASVs.merge(clades_ASVs_df, how='left', left_on='ASV',right_on='ASVs')
allASVs_HRclade_codiv = allASVs_HRclade.merge(blast_res_pass, how='left', on='ASV')
allASVs_HRclade_codiv.head()

#add taxonomy
allASVs_HRclade_codiv['Family'] = allASVs_HRclade_codiv['ASV'].apply(lambda ASV: tax_fam_dict[ASV])
allASVs_HRclade_codiv['Genus'] = allASVs_HRclade_codiv['ASV'].apply(lambda ASV: tax_gen_dict[ASV])

#add captive sp and site info
metadata['Description_site'] = metadata['Description']+'_' +metadata['site_code']
sample_type_site_dict = dict(zip(metadata['X.SampleID'], metadata['Description_site']))

allASVs_HRclade_codiv['sampleNames'] = allASVs_HRclade_codiv['ASV'].apply(lambda x: ASV_sampleName_dict[x])

description_df = allASVs_HRclade_codiv['sampleNames'].apply(lambda l: pd.Series(
    [sample_type_site_dict[name] for name in l]).value_counts())
description_df = description_df.fillna(0) 
description_cp_df= description_df[['captive_chimp_HOUZ','captive_gorilla_HOUZ','captive_orangutan_HOUZ',
'captive_bonobo_COLZ','captive_gorilla_COLZ','captive_orangutan_COLZ','captive_chimp_PC']]
allASVs_HRclade_codiv_cp = allASVs_HRclade_codiv.merge(description_cp_df, left_index=True,right_index=True)

allASVs_HRclade_codiv_cp.to_csv('analyses/codiv_moeller_ASVs/full_tree_ASV_table.txt',sep='\t',index=False)
print(allASVs_HRclade_codiv_cp.head())

7148
  cladeName                                        sampleNames  sampleNum  \
0  clade_41  [Obregon-TitoAJ_2015__SM41, CosteaPI_2017__SID...        788   
1  clade_41  [Obregon-TitoAJ_2015__SM41, CosteaPI_2017__SID...        788   
2  clade_41  [Obregon-TitoAJ_2015__SM41, CosteaPI_2017__SID...        788   
3  clade_41  [Obregon-TitoAJ_2015__SM41, CosteaPI_2017__SID...        788   
4  clade_41  [Obregon-TitoAJ_2015__SM41, CosteaPI_2017__SID...        788   

   ASVsNum HR_sampleTypes  HR_sampleNum HR_cat   HR_type  CP_pres  \
0      366        [human]           788     HR  HR_human    False   
1      366        [human]           788     HR  HR_human    False   
2      366        [human]           788     HR  HR_human    False   
3      366        [human]           788     HR  HR_human    False   
4      366        [human]           788     HR  HR_human    False   

  CP_sampleTypes  CP_sampleNum      ASVs  
0             []             0  ASV_2623  
1             []             0 